In [1]:
# Necessary packages

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timedelta

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Loading Amazon Homepage
url = "https://www.amazon.in"
driver = webdriver.Chrome()
driver.get(url)

In [3]:
# Our Search keys
search_items = 	['Lipstick for women','Shampoo for women','Face wash','Baby Shampoo','Face Serum']

# List to store our products
products = []   
check = 0

In [4]:
# Function to parse necessary tags from html extracted from driver
def fetch(soup, si, p_no, ogr, sr): 
    print("passed")
    soup = BeautifulSoup(driver.page_source, 'lxml')
    items = soup.find_all("div", {'data-component-type':"s-search-result"})
    for item in items:
        i = ogr
        j = sr
        title = item.find('span', class_="a-size-base-plus a-color-base a-text-normal").text        
        rating = item.find('i')
        total_rating = item.find('span', class_="a-size-base s-underline-text")
        if(rating):
            rating=rating.text
        if(total_rating):
            total_rating.text
        price = item.find('span', class_="a-price-whole")
        if(price != None):
            price = item.find('span', class_="a-price-whole").text
    
        
        m = item.find('div', class_="a-section aok-inline-block")
        if(m!=None):
            mrp = m.find('span',class_="a-offscreen").text
   
        today = datetime.today().date() 
        present = today.strftime("%a")
        tat = None

        # Delivery date and TAT calculation using date manipulation algorithm
        del_day = item.find('span', class_="a-color-base a-text-bold")
        weekday_to_num = {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5, 'Sun': 6}
        if(del_day == None):
            del_day = "NA"
        else:       
            pattern = r"Tomorrow"
            pattern2 = r"Today"
            match = re.search(pattern, del_day.text ,re.IGNORECASE)
            match2 = re.search(pattern2, del_day.text ,re.IGNORECASE)
            if(match):
                del_day = today + timedelta(days=1)
                tat = 1
            elif(match2):
                del_day = today
                tat = 0
            else:
                string = del_day.text
                w_day = string.split()
                
                if w_day:
                    first_word = w_day[0]
                    dod_weekday = first_word.replace(" ", "")
                    dod_weekday= dod_weekday.strip(',')
                    dod_weekday = dod_weekday[:3]
                    weekday_num = weekday_to_num[dod_weekday.capitalize()]
                    present = weekday_to_num[present.capitalize()]
                    tat = abs(present - weekday_num)
                     

        # Quantities bought last month extraction using string regex
        quantities_blm = item.find('span', class_="a-size-base a-color-secondary")
        if (quantities_blm == None):
            quantities_blm = "NA"
        else:  
            pattern = r"bought in past month"
            match = re.search(pattern, quantities_blm.text ,re.IGNORECASE) 
            if(match == None):
                quantities_blm = "NA"
            else:
                qua = quantities_blm.text
                quantities_blm = re.search(r'\d+', qua).group()
                print(quantities_blm)                
        
        type = item.find('span', class_="puis-label-popover-default")

        # Tracing ranking of each product
        if(type == None):
            type = 'Organic'
            ogr += 1
        else:
            type = type.text
            sr += 1
    
        if((i - ogr) == 0):
            og_rank = 0
        else:
            og_rank = ogr
        if((j - sr) == 0):
            s_rank = 0
        else:
            s_rank = sr 
         
        
        deal_tag = item.find('span', class_="a-badge-text")
        if(deal_tag == None):
            deal_tag == 'No Deal'
        else:
            deal_tag = item.find('span', class_="a-badge-text").text

        label = item.find('span', class_='a-badge-label')
        if (label):
            label = label.text

        page_no = int(soup.find('span', class_="s-pagination-item s-pagination-selected").text)
        
        # Storing product attributes in a dictionary
        product ={"Product Title" : title,             
                 "Selling Price" : price,
                 "MRP" : mrp,
                  "Label" : label,
                 "Avg. Star Rating" : rating,
                 "Total Rating" : total_rating,
                 "Quantities_lastmonth" : quantities_blm,
                 "Date of Delivery" : del_day,      
                  "Deal" : deal_tag,            
                  "Delivery TAT" : tat,
                  "Page No." : int(page_no),
                  "Listing Type" : type,
                  "Organic Search Rank" : og_rank,
                  "Sponsored Search Rank" : s_rank,
                  "Search Keyword" : si
                  }
        products.append(product)

    
    # Condition for loading till page 3
    if(p_no < 2):
        ep = a_tags[p_no].get('href')
        link = f"https://www.amazon.in/{ep}"
        driver.get(link)
        print(link)
        p_no += 1
        driver.implicitly_wait(15)
        next_soup = BeautifulSoup(driver.page_source, 'lxml')
        fetch(next_soup, si, p_no, ogr, sr)
    else:
        pass

for si in search_items: 
    print(si)
    driver.get(url)
    search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
    search_button = driver.find_element(By.ID, 'nav-search-submit-button')
    
    driver.implicitly_wait(15)
    
    search_box.send_keys(si)
    search_button.click()
    
    # Wait to load the page
    driver.implicitly_wait(15)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    strip = soup.find("span",'s-pagination-strip')
    
    # Collect urls for next pages
    a_tags = strip.find_all('a')

    # Ogr = Organic Search Rank and Sr Search rank to monitor rankings of every product
    fetch(soup,si,p_no=0,ogr=0,sr=0)

Lipstick for women
passed
600
200
50
4
1
1
2
1
400
1
100
1
300
2
600
800
400
800
800
100
400
2
1
400
50
500
100
50
1
300
4
300
100
100
700
1
50
500
100
100
100
200
300
500
100
100
500
100
100
50
https://www.amazon.in//s?k=Lipstick+for+women&page=2&xpid=ysdnC1X3KYBM2&crid=295RHEU006WNV&qid=1714547268&sprefix=%2Caps%2C195&ref=sr_pg_2
passed
100
500
200
500
50
100
200
100
300
500
600
100
200
300
100
50
50
100
100
100
800
100
100
1
300
50
400
200
200
200
100
50
50
50
50
400
https://www.amazon.in//s?k=Lipstick+for+women&page=3&xpid=ysdnC1X3KYBM2&crid=295RHEU006WNV&qid=1714547268&sprefix=%2Caps%2C195&ref=sr_pg_3
passed
50
50
200
300
100
100
50
100
50
50
100
100
50
300
100
100
50
50
100
300
200
50
50
50
300
Shampoo for women
passed
1
10
8
10
10
10
8
300
4
3
5
2
1
6
9
600
4
400
6
9
6
6
4
10
700
3
4
3
5
300
3
2
1
2
10
1
2
1
5
800
1
900
4
3
2
800
1
1
2
1
3
2
1
1
2
10
2
https://www.amazon.in//s?k=Shampoo+for+women&page=2&xpid=poW8bWkT2BpRK&crid=USHDDS7OI9JX&qid=1714547290&sprefix=%2Caps%2C185&ref

In [5]:
df = pd.json_normalize(products)

In [6]:
df

,Product Title,Selling Price,MRP,Label,Avg. Star Rating,Total Rating,Quantities_lastmonth,Date of Delivery,Deal,Delivery TAT,Page No.,Listing Type,Organic Search Rank,Sponsored Search Rank,Search Keyword
0,"Maybelline New York Liquid Matte Lipstick, Lon...",559,₹650,None,4.2 out of 5 stars,"[25,875]",600,2024-05-02,None,1.0,1,Sponsored,0,1,Lipstick for women
1,Maybelline Superstay Matte Ink Brooklyn Blush ...,559,₹699,None,4.2 out of 5 stars,"[77,083]",200,2024-05-02,None,1.0,1,Sponsored,0,2,Lipstick for women
2,SUGAR Cosmetics Matte Attack Lipstick for Wome...,628,₹749,Limited time deal,3.9 out of 5 stars,"[2,026]",50,2024-05-02,Limited time deal,1.0,1,Sponsored,0,3,Lipstick for women
3,RENEE Madness Ph Lipstick 3gm - Black Lipstick...,338,₹499,Limited time deal,4.1 out of 5 stars,"[8,809]",4,2024-05-02,Limited time deal,1.0,1,Sponsored,0,4,Lipstick for women
4,"Maybelline New York Matte Lipstick, Intense Co...",246,₹329,Best seller,4.1 out of 5 stars,"[64,282]",1,2024-05-02,Best seller,1.0,1,Organic,1,0,Lipstick for women
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,Underated Salicylic Acid 2% Betaine 3% Serum P...,239,₹349,None,4.0 out of 5 stars,[447],1,2024-05-02,None,1.0,3,Organic,144,0,Face Serum
889,Healthyr-U Pristine Face Serum for Radiance & ...,599,₹725,Limited time deal,5.0 out of 5 stars,[4],NA,[Friday 3 May],Limited time deal,2.0,3,Sponsored,0,31,Face Serum
890,SkinInspired Aqua Pump 10% Niacinamide Face Se...,"1,424","₹1,499",Limited time deal,4.8 out of 5 stars,[65],NA,2024-05-02,Limited time deal,1.0,3,Sponsored,0,32,Face Serum
891,Nature4Nature Revitalizer Night Natural Face S...,"1,199","₹2,999",None,4.6 out of 5 stars,[110],50,2024-05-02,None,1.0,3,Sponsored,0,33,Face Serum


In [7]:
df.drop_duplicates(inplace=True)
df = df.explode('Total Rating').reset_index(drop=True)

In [9]:
df['MRP'] = df['MRP'].str.replace('₹', '').str.replace(',', '')

In [15]:
driver.close()
df.to_csv('assignment_GGG.csv', index=False)